<a href="https://colab.research.google.com/github/Mahdi-Miri/Signal_Proccesing/blob/main/Signal_Proccesing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Data Readig

In [ ]:
!pip install obspy

##Modelling

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Conv2D, BatchNormalization, ReLU, MaxPooling2D,
    GlobalAveragePooling2D, TimeDistributed, Dense, Permute, Reshape, Layer
)
from tensorflow.keras.models import Model
import numpy as np

In [ ]:
# --- Custom Layer Definition with Functional Implementation ---
# This layer now has a functional implementation based on the
# Auto-Correlation mechanism using Fast Fourier Transform (FFT).
class CustomAutocorrelationLayer(Layer):
    """
    A custom layer that calculates the temporal auto-correlation of an input tensor.
    It operates on the last axis, which is assumed to be the time dimension.
    The calculation is performed efficiently in the frequency domain using FFT.
    """
    def __init__(self, **kwargs):
        super(CustomAutocorrelationLayer, self).__init__(**kwargs)

    def call(self, inputs):
        # The input shape is expected to be (Batch, Height, Width, TimeSteps)
        # We need to compute auto-correlation along the TimeSteps axis (axis=-1).

        # Get the length of the time series
        sequence_length = tf.shape(inputs)[-1]

        # --- Step 1: Go to Frequency Domain using FFT ---
        # Perform Fast Fourier Transform. tf.signal.rfft is used for real-valued inputs.
        # The length of the FFT is padded to the next power of 2 for efficiency,
        # but for simplicity, we'll use the original length here.
        fft_result = tf.signal.rfft(inputs, fft_length=[sequence_length])

        # --- Step 2: Compute Power Spectral Density ---
        # This is where the correlation is calculated.
        # It's done by multiplying the FFT result by its complex conjugate.
        # This is equivalent to convolution in the time domain.
        power_spectral_density = fft_result * tf.math.conj(fft_result)

        # --- Step 3: Go back to Time Domain using Inverse FFT ---
        # Perform Inverse Real Fast Fourier Transform to get the auto-correlation series.
        autocorr_result = tf.signal.irfft(power_spectral_density, fft_length=[sequence_length])

        return autocorr_result

    def get_config(self):
        # Required for model saving and loading
        config = super(CustomAutocorrelationLayer, self).get_config()
        return config

# --- Full Architecture Definition ---
def build_spatio_temporal_model(input_shape=(60, 60, 100)):
    """
    Builds the complete Spatio-Temporal feature extraction model based on the diagram.

    Args:
        input_shape (tuple): The shape of the input data (Height, Width, TimeSteps).

    Returns:
        keras.Model: The compiled Keras model.
    """
    # Define the input layer for our spatio-temporal data
    inputs = Input(shape=input_shape)

    # --- Part 1: Temporal Transformation & Reshaping ---

    # Apply the functional autocorrelation layer to find temporal patterns.
    # This layer processes each spatial point's time series (60x60 series of length 100).
    # Output values now represent the strength of temporal correlations.
    temporal_features = CustomAutocorrelationLayer()(inputs)  # Shape: (None, 60, 60, 100)

    # Permute the dimensions to bring the time-steps to the front for the next stage.
    # (Height, Width, TimeSteps) -> (TimeSteps, Height, Width)
    permuted = Permute((3, 1, 2))(temporal_features)  # Shape: (None, 100, 60, 60)

    # Reshape to add a 'channels' dimension. Each time-step is now a 60x60x1 image,
    # ready to be processed by a 2D CNN.
    reshaped = Reshape((100, 60, 60, 1))(permuted)  # Shape: (None, 100, 60, 60, 1)

    # --- Part 2: Time-Distributed Spatial Feature Extraction ---

    # Define the CNN block that extracts spatial features from a single 60x60 frame.
    cnn_block = tf.keras.Sequential([
        # Finds low-level features like edges and gradients.
        Conv2D(32, kernel_size=(3, 3), padding='same'),
        BatchNormalization(),
        ReLU(),
        # Downsamples the feature map to make representations more robust.
        MaxPooling2D(pool_size=(2, 2)),

        # Finds higher-level features by combining low-level ones.
        Conv2D(64, kernel_size=(3, 3), padding='same'),
        BatchNormalization(),
        ReLU(),
        MaxPooling2D(pool_size=(2, 2)),

        # Summarizes all spatial features in the frame into a single, fixed-size vector.
        GlobalAveragePooling2D()
    ], name='internal_cnn_block')

    # Apply the defined cnn_block to each of the 100 time-steps independently.
    # The output is a sequence of 100 feature vectors, one for each time-step.
    time_distributed_cnn = TimeDistributed(cnn_block)(reshaped)  # Shape: (None, 100, 64)

    # --- Part 3: Final Transformation ---

    # A Dense layer refines the features from the CNN, reducing each feature vector's size to 10.
    # It learns combinations of the spatial features.
    dense_features = Dense(10, activation='relu')(time_distributed_cnn)  # Shape: (None, 100, 10)

    # Final permutation to get the output shape of (Features, TimeSteps).
    # This format might be useful for downstream tasks that analyze each feature over time.
    final_output = Permute((2, 1))(dense_features)  # Shape: (None, 10, 100)

    # Create the final model by defining its inputs and outputs
    model = Model(inputs=inputs, outputs=final_output, name='SpatioTemporal_Feature_Extractor')

    return model

# --- Model Creation, Compilation, and Execution Example ---

if __name__ == '__main__':
    # Define the input shape as per the diagram
    INPUT_SHAPE = (60, 60, 100)

    # Build the model
    model = build_spatio_temporal_model(input_shape=INPUT_SHAPE)

    # Compile the model. A loss function and optimizer are needed for training.
    # For feature extraction, you might not train it, but compilation is a good practice.
    model.compile(optimizer='adam', loss='mse')

    # Print the model summary to verify the architecture and shapes
    print("--- Model Summary ---")
    model.summary()

    # --- Example of running the model with dummy data ---
    print("\n--- Running a test prediction ---")
    # Create a batch of 2 dummy "videos" with random data
    dummy_data = np.random.rand(2, *INPUT_SHAPE)

    # Get the model's prediction
    predictions = model.predict(dummy_data)

    # Print the shape of the output to confirm it matches the design
    print(f"Input data shape: {dummy_data.shape}")
    print(f"Final output shape: {predictions.shape}")


--- Model Summary ---


Model: "SpatioTemporal_Feature_Extractor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 60, 60, 100)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ custom_autocorrelation_layer    │ (None, None, None,     │             0 │
│ (CustomAutocorrelationLayer)    │ 100)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ permute (Permute)               │ (None, 100, None,      │             0 │
│                                 │ None)                  │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 100, 60, 60, 1) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 100, 64)        │        19,200 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100, 10)        │           650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ permute_1 (Permute)             │ (None, 10, 100)        │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,850 (77.54 KB)

 Trainable params: 19,658 (76.79 KB)

 Non-trainable params: 192 (768.00 B)


--- Running a test prediction ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step
Input data shape: (2, 60, 60, 100)
Final output shape: (2, 10, 100)


##testing

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Input, Permute, Reshape, Conv2D, BatchNormalization, ReLU, MaxPooling2D, GlobalAveragePooling2D, TimeDistributed, Dense
from tensorflow.keras.models import Model
import obspy
from obspy import UTCDateTime, read, read_inventory, Stream, Trace
from scipy.signal import correlate
import matplotlib.pyplot as plt
import seaborn as sns

# ==============================================================================
#  1. KERAS MODEL DEFINITION
# ==============================================================================
class CustomAutocorrelationLayer(Layer):
    """Custom layer to calculate temporal auto-correlation using FFT."""
    def __init__(self, **kwargs):
        super(CustomAutocorrelationLayer, self).__init__(**kwargs)
    def call(self, inputs):
        sequence_length = tf.shape(inputs)[-1]
        fft_result = tf.signal.rfft(inputs, fft_length=[sequence_length])
        power_spectral_density = fft_result * tf.math.conj(fft_result)
        autocorr_result = tf.signal.irfft(power_spectral_density, fft_length=[sequence_length])
        return autocorr_result

def build_spatio_temporal_model(input_shape=(60, 60, 100)):
    """Builds the complete Spatio-Temporal feature extraction model."""
    inputs = Input(shape=input_shape)
    # The custom layer is given a name to access its output later
    temporal_features = CustomAutocorrelationLayer(name='autocorr_layer')(inputs)
    permuted = Permute((3, 1, 2))(temporal_features)
    reshaped = Reshape((100, 60, 60, 1))(permuted)
    cnn_block = tf.keras.Sequential([
        Conv2D(32, (3, 3), padding='same'), BatchNormalization(), ReLU(), MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), padding='same'), BatchNormalization(), ReLU(), MaxPooling2D((2, 2)),
        GlobalAveragePooling2D()
    ], name='internal_cnn_block')
    time_distributed_cnn = TimeDistributed(cnn_block)(reshaped)
    dense_features = Dense(10, activation='relu')(time_distributed_cnn)
    final_output = Permute((2, 1))(dense_features)
    model = Model(inputs=inputs, outputs=final_output, name='SpatioTemporal_Feature_Extractor')
    return model

# ==============================================================================
#  2. DATA PREPARATION FUNCTION
# ==============================================================================
def prepare_ambient_noise_data(recordings_path, xml_path, station_list, num_days, shape):
    """Loads data, calculates daily autocorrelations, and formats for the model."""
    height, width, timesteps = shape
    all_autocorrelations = []
    print("--- Starting Data Preparation ---")
    try:
        stream = read(recordings_path)
        inventory = read_inventory(os.path.join(xml_path, "GH.*.xml"))
        start_time = stream[0].stats.starttime
    except Exception as e:
        print(f"ERROR: Could not load data files. Reason: {e}")
        return None
    for station_name in station_list:
        print(f"Processing Station: {station_name}")
        station_stream = stream.select(station=station_name)
        if not station_stream:
            print(f"  - No data found for station {station_name}. Skipping.")
            continue
        for day in range(num_days):
            day_start = start_time + day * 86400
            st_day = station_stream.slice(starttime=day_start, endtime=day_start + 86400)
            if not st_day:
                continue
            try:
                # --- Pre-processing Steps with all fixes ---
                st_day.detrend('linear')
                st_day.taper(max_percentage=0.05, type="hann")
                # Use robust keyword arguments
                st_day.remove_response(
                    inventory=inventory,
                    output="VEL",
                    pre_filt=(0.01, 0.05, 45, 50)
                )
                # Use stable, multi-stage decimation
                st_day.decimate(factor=10)
                st_day.decimate(factor=10)
                st_day.filter('bandpass', freqmin=0.1, freqmax=0.4, zerophase=True)
                # Use correct one-bit normalization
                for trace in st_day:
                    trace.data = np.sign(trace.data)

                # --- Autocorrelation ---
                trace = st_day[0]
                npts = trace.stats.npts
                autocorr_data = correlate(trace.data, trace.data, mode='full')[npts-1:]

                # --- Finalize Data for Model ---
                if len(autocorr_data) > timesteps:
                    final_data = autocorr_data[:timesteps]
                else:
                    padding = np.zeros(timesteps - len(autocorr_data))
                    final_data = np.concatenate([autocorr_data, padding])
                all_autocorrelations.append(final_data)
            except Exception as e:
                print(f"    - Error on day {day+1} for {station_name}: {type(e).__name__} - {e}")
                continue
    if not all_autocorrelations:
        print("ERROR: No data was successfully processed.")
        return None

    stacked_signals = np.array(all_autocorrelations)
    model_input_data = np.zeros((stacked_signals.shape[0], height, width, timesteps))
    model_input_data[:, height // 2, width // 2, :] = stacked_signals
    return model_input_data

# ==============================================================================
#  3. MAIN EXECUTION BLOCK
# ==============================================================================
if __name__ == "__main__":
    # --- CONFIGURATION: Edit these paths ---
    RECORDED_DATA_FILE = "/content/drive/MyDrive/Signal/2013-01-01-00-00-00.mseed"
    XML_FOLDER_PATH = "/content/drive/MyDrive/Signal/"
    STATION_NAMES = ['AKOS', 'KLEF', 'KUKU', 'MRON']
    NUM_DAYS_TO_PROCESS = 1# Number of days to process for each station
    MODEL_SHAPE = (60, 60, 100)
    NEW_SAMPLING_RATE = 1.0 # Hz

    # --- Step 1: Prepare the data ---
    prepared_data = prepare_ambient_noise_data(
        recordings_path=RECORDED_DATA_FILE, xml_path=XML_FOLDER_PATH,
        station_list=STATION_NAMES, num_days=NUM_DAYS_TO_PROCESS,
        shape=MODEL_SHAPE
    )

    if prepared_data is not None:
        print(f"\n✅ Data preparation complete. Input shape: {prepared_data.shape}")

        # --- Step 2: Build and Compile Keras Model ---
        full_model = build_spatio_temporal_model(input_shape=MODEL_SHAPE)
        full_model.compile(optimizer='adam', loss='mse')

        # --- Step 3: Run Visualizations ---
        print("\n--- Creating Visualizations ---")

        # VISUALIZATION 1: Heatmap of Final Model Features
        print("Running full model to generate feature heatmap...")
        predictions = full_model.predict(prepared_data)

        # Visualize the features for the first sample (e.g., AKOS, Day 1)
        sample_to_visualize = predictions[0]
        plt.figure(figsize=(12, 6))
        sns.heatmap(sample_to_visualize, cmap='viridis')
        plt.title("Heatmap of Final Extracted Features (First Sample)")
        plt.xlabel("Time Steps")
        plt.ylabel("Feature Index")
        plt.show()

        # VISUALIZATION 2: Seismic Section Plot of Raw Autocorrelation
        print("Running intermediate model to generate section plot...")
        intermediate_model = Model(inputs=full_model.input, outputs=full_model.get_layer('autocorr_layer').output)
        autocorr_results_4d = intermediate_model.predict(prepared_data)
        autocorr_results_1d = autocorr_results_4d[:, MODEL_SHAPE[0] // 2, MODEL_SHAPE[1] // 2, :]

        autocorr_stream = Stream()
        for i, daily_result in enumerate(autocorr_results_1d):
            trace = Trace(data=daily_result)
            trace.stats.sampling_rate = NEW_SAMPLING_RATE
            # Set a distance value in meters to satisfy the plot function
            trace.stats.distance = i * 1000
            autocorr_stream.append(trace)

        print("Displaying section plot...")
        autocorr_stream.plot(
            type='section',
            recordlength=20,
            fillcolors=('red', 'black'),
            stack_results=True,
            title="Raw Autocorrelation from Keras Layer"
        )